<a href="https://colab.research.google.com/github/dimsparagis0210/DFU-using-Deep-Learning/blob/main/Faster_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Faster RCNN Model Development

#### The goal of this BSc thesis is to develop a Deep Learning model for Object Detection. Specifically, the object of the model will be to detect ulcers in Diabetic Foots (DFU)

## Getting Dataset from Google drive

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# Define dataset path (adjust if needed)
dataset = '/content/drive/MyDrive/DFU_Data_Faster_RCNN'
trainset = dataset + '/images/train'
testset = dataset + '/images/test'
validset = dataset + '/images/valid'

# List categories (folders)
os.listdir('/content/drive/MyDrive')

['Colab Notebooks',
 'ML',
 'Machine Learning A-Z (Model Selection)',
 'Deep Learning',
 'ML_task',
 'Screencastify',
 'DFU',
 'IoT, Big Data, AI Smart Farms.gdoc',
 'Conceptual Analysis of the Study on Environmentally Sustainable Smart Cities.gdoc',
 'DFU_Data',
 'Stergiou CV (1).gdoc',
 'DFU_Data_Tensorflow',
 'Ενοποιημένες',
 'DFU_Data_Faster_RCNN',
 'SSD (1).ipynb',
 'Faster-RCNN.ipynb',
 'SSD.ipynb']

In [ ]:
pip install ray[tune]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 MB 34.2 MB/s eta 0:00:00


In [2]:
import json
import numpy as np
import tensorflow as tf
import cv2
import albumentations as A
from pycocotools.coco import COCO
import torch
import torchvision
from torch.utils.data import DataLoader
from ray import tune
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Data Pipeline

In [4]:
import json

# Load the annotation file
with open('/content/drive/MyDrive/DFU_Data_Faster_RCNN/annotations/train.coco.json') as f:
    data = json.load(f)

# Extract category IDs and names
categories = data.get('categories', [])
num_classes = len(categories)
class_names = [category['name'] for category in categories]

print(f"Number of classes: {num_classes}")
print("Class names:", class_names)
len(class_names)

Number of classes: 2
Class names: ['objects', 'Ulcer']


2

In [ ]:
import os
import torch
from PIL import Image
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader

import os
import torch
from PIL import Image
from pycocotools.coco import COCO
import torchvision.transforms as T

class UlcerDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation_file, transforms=None):
        self.root = root
        self.coco = COCO(annotation_file)
        self.ids = list(self.coco.imgs.keys())
        self.transforms = transforms

    def __getitem__(self, index):
      coco = self.coco
      img_id = self.ids[index]
      ann_ids = coco.getAnnIds(imgIds=img_id)
      anns = coco.loadAnns(ann_ids)
      path = coco.loadImgs(img_id)[0]['file_name']

      img = Image.open(os.path.join(self.root, path)).convert('RGB')

      boxes = []
      labels = []
      for ann in anns:
          xmin = ann['bbox'][0]
          ymin = ann['bbox'][1]
          xmax = xmin + ann['bbox'][2]
          ymax = ymin + ann['bbox'][3]
          boxes.append([xmin, ymin, xmax, ymax])
          labels.append(ann['category_id'])

      if len(boxes) == 0:
          # Handle the case where no annotations are present
          boxes = torch.zeros((0, 4), dtype=torch.float32)
          labels = torch.zeros((0,), dtype=torch.int64)
      else:
          boxes = torch.as_tensor(boxes, dtype=torch.float32)
          labels = torch.as_tensor(labels, dtype=torch.int64)

      image_id = torch.tensor([img_id])
      area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
      iscrowd = torch.zeros((len(anns),), dtype=torch.int64)

      target = {
          'boxes': boxes,
          'labels': labels,
          'image_id': image_id,
          'area': area,
          'iscrowd': iscrowd
      }

      if self.transforms:
          img = self.transforms(img)

      return img, target


    def __len__(self):
        return len(self.ids)

    @staticmethod
    def collate_fn(batch):
        return tuple(zip(*batch))

    from ray import tune

def train_faster_rcnn(config, dataset=None):
    if dataset is None:
        raise ValueError("A preprocessed dataset must be provided.")

    data_loader = DataLoader(
        dataset,
        batch_size=config["batch_size"],
        shuffle=True,
        num_workers=4,
        collate_fn=UlcerDataset.collate_fn
    )

    weights = torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=weights)

    num_classes = len(class_names)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = torch.optim.SGD(
        [p for p in model.parameters() if p.requires_grad],
        lr=config["lr"],
        momentum=config["momentum"],
        weight_decay=config["weight_decay"]
    )

    for epoch in range(config["num_epochs"]):
        model.train()
        total_loss = 0
        for images, targets in data_loader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            total_loss += losses.item()

        avg_loss = total_loss / len(data_loader)
        tune.report(loss=avg_loss)



## Data Preprocessing

### Data Augmentation & Normalization

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.RandomRotation(degrees=15))
        transforms.append(T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1))
    transforms.append(T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    return T.Compose(transforms)

### Apply preprocessing steps to the trainset

In [ ]:
from torch.utils.data import DataLoader

train_dataset = UlcerDataset(root= trainset, annotation_file=os.path.join('/content/drive/MyDrive/DFU_Data_Faster_RCNN', 'annotations', 'train.coco.json'), transforms=get_transform(train=True))
test_dataset = UlcerDataset(root= testset, annotation_file= os.path.join('/content/drive/MyDrive/DFU_Data_Faster_RCNN', 'annotations', 'test.coco.json'), transforms=get_transform(train=False))
valid_dataset = UlcerDataset(root= validset, annotation_file= os.path.join('/content/drive/MyDrive/DFU_Data_Faster_RCNN', 'annotations', 'valid.coco.json'), transforms=get_transform(train=True))

# Create data loaders
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


## Model Development

### Hyperparameter Optimization

In [ ]:
from ray import tune

config = {
    "lr": tune.loguniform(1e-5, 1e-2),
    "momentum": tune.uniform(0.7, 0.99),
    "weight_decay": tune.loguniform(1e-5, 1e-2),
    "batch_size": tune.choice([2, 4, 8]),
    "num_epochs": 10,
}

In [ ]:
from ray import tune
from ray.tune.schedulers import ASHAScheduler

# Define the dataset
dataset = UlcerDataset(root = trainset, annotation_file=os.path.join('/content/drive/MyDrive/DFU_Data_Faster_RCNN', 'annotations', 'train.coco.json'), transforms=get_transform(train=True))

# Define the scheduler
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=100,
    grace_period=1,
    reduction_factor=2
)

# Run the hyperparameter search
analysis = tune.run(
    tune.with_parameters(UlcerDataset.train_faster_rcnn, dataset=dataset),
    resources_per_trial={"cpu": 8, "gpu": 1},
    config=config,
    num_samples=10,
    scheduler=scheduler,
)

# Get the best hyperparameters
best_config = analysis.best_config
print("Best hyperparameters found were: ", best_config)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
+--------------------------------------------------------------------------+
| Configuration for experiment     train_faster_rcnn_2025-03-18_14-43-31   |
+--------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                   |
| Scheduler                        AsyncHyperBandScheduler                 |
| Number of trials                 10                                      |
+--------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_faster_rcnn_2025-03-18_14-43-31
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-03-18_13-59-26_187931_461/artifacts/2025-03-18_14-43-31/train_faster_rcnn_2025-03-18_14-43-31/driver_artifacts`

Trial status: 10 PENDING
Current time: 2025-03-18 14:43:31. Total running time: 0s
Logi

(train_faster_rcnn pid=15997) Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers). Got range [-2.117904..2.64].



Trial status: 1 RUNNING | 9 PENDING
Current time: 2025-03-18 14:44:01. Total running time: 30s
Logical resource usage: 8.0/8 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------------------------------------------+
| Trial name                      status              lr     momentum     weight_decay     batch_size |
+-----------------------------------------------------------------------------------------------------+
| train_faster_rcnn_5c9b3_00000   RUNNING    0.000144045     0.702534      0.000104817              4 |
| train_faster_rcnn_5c9b3_00001   PENDING    0.000167215     0.700716      0.00148171               4 |
| train_faster_rcnn_5c9b3_00002   PENDING    0.000348297     0.905562      0.00776717               4 |
| train_faster_rcnn_5c9b3_00003   PENDING    1.90036e-05     0.791761      3.9215e-05               8 |
| train_faster_rcnn_5c9b3_00004   PENDING    6.17201e-05     0.854702      0.00306342              

2025-03-18 15:04:06,806	ERROR tune_controller.py:1331 -- Trial task failed for trial train_faster_rcnn_5c9b3_00000
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/worker.py", line 2771, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ray/_private/worker.p


Trial train_faster_rcnn_5c9b3_00000 errored after 0 iterations at 2025-03-18 15:04:06. Total running time: 20min 35s
Error file: /tmp/ray/session_2025-03-18_13-59-26_187931_461/artifacts/2025-03-18_14-43-31/train_faster_rcnn_2025-03-18_14-43-31/driver_artifacts/train_faster_rcnn_5c9b3_00000_0_batch_size=4,lr=0.0001,momentum=0.7025,weight_decay=0.0001_2025-03-18_14-43-31/error.txt

Trial train_faster_rcnn_5c9b3_00001 started with configuration:
+--------------------------------------------------------+
| Trial train_faster_rcnn_5c9b3_00001 config             |
+--------------------------------------------------------+
| batch_size                                           4 |
| lr                                             0.00017 |
| momentum                                       0.70072 |
| num_epochs                                          10 |
| weight_decay                                   0.00148 |
+--------------------------------------------------------+


...

(train_faster_rcnn pid=21355) Figure(640x480)

Trial status: 1 ERROR | 1 RUNNING | 8 PENDING
Current time: 2025-03-18 15:04:33. Total running time: 21min 1s
Logical resource usage: 8.0/8 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------------------------------------------+
| Trial name                      status              lr     momentum     weight_decay     batch_size |
+-----------------------------------------------------------------------------------------------------+
| train_faster_rcnn_5c9b3_00001   RUNNING    0.000167215     0.700716      0.00148171               4 |
| train_faster_rcnn_5c9b3_00002   PENDING    0.000348297     0.905562      0.00776717               4 |
| train_faster_rcnn_5c9b3_00003   PENDING    1.90036e-05     0.791761      3.9215e-05               8 |
| train_faster_rcnn_5c9b3_00004   PENDING    6.17201e-05     0.854702      0.00306342               8 |
| train_faster_rcnn_5c9b3_00005   PEND

2025-03-18 15:05:05,490	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2025-03-18 15:05:05,495	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_faster_rcnn_2025-03-18_14-43-31' in 0.0035s.


Trial status: 1 ERROR | 1 RUNNING | 8 PENDING
Current time: 2025-03-18 15:05:05. Total running time: 21min 34s
Logical resource usage: 8.0/8 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------------------------------------------+
| Trial name                      status              lr     momentum     weight_decay     batch_size |
+-----------------------------------------------------------------------------------------------------+
| train_faster_rcnn_5c9b3_00001   RUNNING    0.000167215     0.700716      0.00148171               4 |
| train_faster_rcnn_5c9b3_00002   PENDING    0.000348297     0.905562      0.00776717               4 |
| train_faster_rcnn_5c9b3_00003   PENDING    1.90036e-05     0.791761      3.9215e-05               8 |
| train_faster_rcnn_5c9b3_00004   PENDING    6.17201e-05     0.854702      0.00306342               8 |
| train_faster_rcnn_5c9b3_00005   PENDING    0.00959921      0.907032      0.0012656

2025-03-18 15:05:15,514	ERROR tune.py:1037 -- Trials did not complete: [train_faster_rcnn_5c9b3_00000]
2025-03-18 15:05:15,515	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2025-03-18 15:05:15,520	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 8 trial(s):
- train_faster_rcnn_5c9b3_00002: FileNotFoundError('Could not fetch metrics for train_faster_rcnn_5c9b3_00002: both result.json and progress.csv were not found at /root/ray_results/train_faster_rcnn_2025-03-18_14-43-31/train_faster_rcnn_5c9b3_00002_2_batch_size=4,lr=0.0003,momentum=0.9056,weight_decay=0.0078_2025-03-18_14-43-31')
- train_faster_rcnn_5c9b3_00003: FileNotFoundError('Could not fetch metrics for train_faster_rcnn_5c9b3_00003: both result.json and progress.csv were not found at /root/ray_results/train_faster_rcnn_2025-03-18_14-43-31/train_faster_rcnn_5c9b3_00003_3_batch_size=8,lr=0.0000,momentum=0.7918,weig

ValueError: To fetch the `best_config`, pass a `metric` and `mode` parameter to `tune.run()`. Alternatively, use the `get_best_config(metric, mode)` method to set the metric and mode explicitly.

In [ ]:
d